In [1]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [2]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Projects: SLIP, SLIP Elementary, SLIP Adolescents, Clinical Imaging Genetics
project = "SLIP Elementary" # Clinical Fetal"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)
    
get_grader_status_report(name)


select * from lab.test_grader_table_with_metadata where grader_name = 'Jenna Schabdach';
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
Jenna Schabdach has graded 38142 unique reports of 38144 assigned where
34245 have been given a grade of 0
1785 have been given a grade of 1
2104 have been given a grade of 2


In [3]:
isOnMainTable = welcome_user(name)

Welcome, Jenna Schabdach
151
151
Jenna Schabdach has graded 151 of 151 reliability reports
You currently have 2 ungraded reports to work on.


In [ ]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        mark_one_report_sql(name, project, toHighlight)
    else:
        mark_selfeval_report_sql(name, toHighlight)


0
SELECT * FROM lab.test_grader_table_with_metadata reports join lab.proc_ord_projects projects on (reports.proc_ord_id = projects.proc_ord_id and reports.pat_id = projects.pat_id)  WHERE grader_name = "Jenna Schabdach" and grade = 999 and grade_category = "Unique" LIMIT 1
Year of scan: 2024
Age at scan: 13.06 years
It does belong to the following cohorts: Clinical Imaging Genetics

Examination: MRI of the brain with and without contrast INDICATION: Dystonia, developmental delay, abnormality of gait and mobility, abnormal MRI of the brain 

COMPARISON: Brain MRI December 20, 2021 

TECHNIQUE: MR imaging of the brain on a 3 Tesla MRI system before and after intravenous contrast administration. Imaging sequences included diffusion-weighted imaging, 3-D T1 MPRAGE pre and postcontrast, axial T1 spin-echo with fat saturation pre and postcontrast, susceptibility weighted imaging, axial and coronal T2, axial and coronal T2 FLAIR with fat saturation, 3D FGATIR sequence, 3D T2 SPACE, ASL perfus

In [5]:
## If/when you run out of reports, you can uncomment the following line of code
#   and run this cell to get 50 more reports from the database. Make sure your
#   values for `project` and `queryFn` match the project you're working on.
# Projects: SLIP Adolescents, SLIP, SLIP Elementary, Clinical Imaging Genetics
# project = "SLIP Adolescents"
# get_second_look_reports_to_grade(name, 5)
get_more_reports_to_grade(name, project, num_to_add=5)

It is expected for this function to take several minutes to run. Your patience is appreciated.
Project: SLIP Elementary
Total reports: 15009 (note each report must be graded by 2 graders)
Graded 0: 805
Graded 1: 2102
Graded 2: 911
Queued: 289
Skipped: 13
insert into lab.test_grader_table_with_metadata (proc_ord_id, grader_name, grade, grade_category, pat_id, age_in_days, proc_ord_year, proc_name, report_origin_table, grade_date) 
        select
          distinct 
          proc_ord.proc_ord_id, "Jenna Schabdach" as grader_name,
          999 as grade,
          "Unique" as grade_category,
          proc_ord.pat_id,
          proc_ord.proc_ord_age as age_in_days,
          proc_ord.proc_ord_year,
          proc_ord.proc_ord_desc as proc_name,
          "arcus.procedure_order" as report_origin_table, "0000-00-00" as grade_date 
        from
          arcus.procedure_order proc_ord
          join arcus.patient pat on proc_ord.pat_id = pat.pat_id
        where
          proc_ord.proc_ord_